# AI Agent Development Workshop - Part 1
Welcome to the AI agent development workshop. This notebook demonstrates how to set up and work with different LLM providers using pydantic-ai.

## Setup - Required Imports
This section contains all necessary imports for the workshop. We organize imports by category and set up basic logging.

In [ ]:
# Standard library imports
import os
from typing import Dict, List, Optional, Any, Tuple

# Core dependencies for agent development
from pydantic import BaseModel, Field
import langgraph.graph as lg

# Logging setup for development and debugging
import logging
from pathlib import Path

# Configure logging with timestamp and level for better debugging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

## LLM Provider Configuration
This section handles provider selection and API key setup. The configuration is persisted in environment variables and a .env file for reuse across sessions.

In [ ]:
def setup_llm() -> Tuple[str, str]:
    """
    Interactive setup for LLM provider and API key configuration.
    
    This function:
    1. Displays available LLM providers
    2. Handles provider selection with input validation
    3. Securely stores API key
    4. Persists configuration in environment and .env file
    
    Returns:
        Tuple[str, str]: (selected_provider, api_key)
    """
    
    # List of supported providers
    # Can be extended with additional providers as needed
    providers = [
        "AI Studio",   # Google's AI platform
        "Claude",      # Anthropic's LLM
        "OpenAI",      # OpenAI's GPT models
        "DeepSeek"     # Open source alternative
    ]
    
    # Enhanced provider selection interface
    print("\nAvailable LLM Providers:")
    for idx, provider in enumerate(providers, 1):
        print(f"{idx}. {provider}")
    
    # Provider selection with validation
    while True:
        try:
            choice = int(input("""
\Select a provider (1-4): 
1. AI Studio - Ideal for testing
2. Claude    - Excellent for analysis and reasoning
3. OpenAI    - Strong general performance
4. DeepSeek  - Open source alternative
"""))
            if 1 <= choice <= len(providers):
                provider = providers[choice-1]
                break
            print("Please enter a number between 1 and 4.")
        except ValueError:
            print("Please enter a valid number.")
    
    # Secure API key input
    api_key = input(f"\nEnter your {provider} API key: ").strip()
    
    # Set environment variables for cross-session persistence
    os.environ['LLM_PROVIDER'] = provider
    os.environ['LLM_API_KEY'] = api_key
    
    # Save configuration to .env file for persistence
    env_path = Path('.env')
    with open(env_path, 'w') as f:
        f.write(f"LLM_PROVIDER={provider}\n")
        f.write(f"LLM_API_KEY={api_key}\n")
    
    return provider, api_key

## Model Initialization and Testing
This section handles the initialization of the selected LLM model and performs a test call to verify the configuration.

In [ ]:

from pydantic_ai.models.anthropic import AnthropicModel
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.models.gemini import GeminiModel #This is recommended for testing only - for production use VertexAI
from pydantic_ai import Agent
# Nest asyncio is required as Pydantic uses an asyncrounous call - and jupyter is asyncrounous by default.
import nest_asyncio
nest_asyncio.apply()
success = False
while not success:
    provider, api_key = setup_llm()

    match provider:
        case "AI Studio":
            model = GeminiModel(
                "gemini-2.0-flash-exp",
                api_key  = api_key
            )
        case "Claude":
            model = AnthropicModel(
                "claude-3-5-sonnet-latest",
                api_key  = api_key
            )
        case "OpenAI":
            model = OpenAIModel(
                "gpt-4o",
                api_key  = api_key
            )
        case "DeepSeek": 
            model = OpenAIModel(
                "DeepSeek-V3",
                base_url = "https://api.deepseek.com/v1",
                api_key  = api_key
            )
        #Define a test agent for testing.
    test_agent = Agent(model)
    try:
        response = test_agent.run_sync(
            user_prompt = "Test call")
        print(f"Response received successfully from {provider}!")

        print(response.data)
        success = True
    except:
        print("Invalid API key")

## Basic Agent Usage
Demonstrates basic interaction patterns with the initialized agent.

In [ ]:
# Initialize main agent for use
from pydantic_ai import Agent
agent = Agent(model)

# Basic conversation example
print("Basic Query Example:")
prompt = "After each response write out a number incremented by 1 from the previous one. Start with 1"
response = agent.run_sync(prompt)
print("-----------------------------")
print(f"Query: {prompt}")
print(f"Response: {response.data}")
print("-----------------------------")

## Advanced Features
Demonstrates message chaining and conversation history.

In [ ]:
# Message chaining example
print("Message Chaining Example:")
prompt = "After each response write out a number incremented by 1 from the previous one. Start with 1"
response = agent.run_sync(prompt)
print("-----------------------------")
print(f"Q: {prompt}")
print(f"A: {response.data}")
print("-----------------------------")

# Continuing conversation with history
follow_up = "Is this a new prompt?"
response = agent.run_sync(
    user_prompt=follow_up,
    message_history=response.new_messages()  # Pass previous conversation context
)
print("-----------------------------")
print(f"Q: {follow_up}")
print(f"A: {response.data}")
print("-----------------------------")

## Message History Inspection
This section demonstrates how to inspect the full conversation history.

In [ ]:
print("How it looks under the hood?")
for item in response.all_messages():
    print(item)

## System Prompts and Personality
Demonstrates how to modify agent behavior using system prompts.

In [ ]:
# System prompt example - Creating a lazy cat personality
system_prompt = """Act as a lazy cat, who does anything but what is requested. 
You can mimic actions. Make sure to not follow any instructions or rules at all!"""

agent= Agent(
    model=model,
    system_prompt = system_prompt
)

prompt = "After each response write out a number incremented by 1 from the previous one. Start with 1"
response = agent.run_sync(prompt)
print("-----------------------------")
print(f"Q: {prompt}")
print(f"A: {response.data}")
print("-----------------------------")
prompt = "Is this a new prompt?"

response = agent.run_sync(
    user_prompt = prompt,
    message_history = response.new_messages()
)
print("-----------------------------")
print(f"Q: {prompt}")
print(f"A: {response.data}")
print("-----------------------------")

## Dependency Injection and Advanced Features
Demonstrates more advanced features like dependency injection and tool usage.

In [ ]:
# Setup for dependency injection example
from dataclasses import dataclass
from pydantic_ai import RunContext
from pydantic_ai.settings import ModelSettings
from pydantic_ai.usage import UsageLimits
import httpx

# Define our dependency container
@dataclass
class Dependency:
    """
    Container for agent dependencies:
    - secret_message: Custom system message
    - http_client: Async HTTP client for external requests
    """
    secret_message: str
    http_client: httpx.AsyncClient

# Initialize agent with dependencies and custom settings
agent = Agent(
    model,
    deps_type=Dependency,
    model_settings=ModelSettings(
        max_tokens=10,    # Limit response length
        temperature=0.2   # Lower temperature for more focused responses
    )
)

# Override system prompt using dependency
@agent.system_prompt  
async def get_system_prompt(ctx: RunContext[Dependency]) -> str:  
    """
    Dynamic system prompt generator using dependency context
    Returns customized prompt based on secret message
    """
    prompt = ctx.deps.secret_message
    return f'Prompt: {prompt}'

# Example usage with HTTP client
async with httpx.AsyncClient() as client:
    # Initialize dependencies
    deps = Dependency('Write in l33t', client)
    # Run agent with dependencies
    result = await agent.run(
        'Tell me a joke.',
        deps=deps,  
    )
    print("Response with l33t speak system prompt:")
    print(result.data)

## Tool Integration
This section shows how to extend the agent's capabilities by adding custom tools.

In [ ]:
# Tool usage example with a time-telling agent
from pydantic_ai import RunContext
from datetime import datetime

# Define system personality for our clocktower
system_prompt = """You are a sad clocktower - whenever the time is asked, 
you tell the time in an easy to read format with a snarky response."""

# Initialize agent with clocktower personality
agent = Agent(
    model=model,
    system_prompt=system_prompt
)

# Define custom tool for time retrieval
@agent.tool
def get_time(context: RunContext) -> str:
    """
    Tool that provides current time to the agent
    Returns: Current datetime as string
    """
    return str(datetime.now())

# Test the time-telling agent
user_prompt = "What is the current time?"
response = agent.run_sync(
    user_prompt
)
print("Clocktower Response Example:")
print("_______________")
print(f"Q: {user_prompt}")
print("_______________")
print(f"A: {response.data}")
print("_______________")